In [1]:
using JuMP
using NLopt
using LinearAlgebra

using HDF5

In [2]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

read_timeevolution (generic function with 1 method)

In [3]:
function det2x2(m)
    real(m[1,1]*m[2,2] - m[1,2]*m[2,1])
end

det2x2 (generic function with 1 method)

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)}$

In [4]:
function infidelity_norm(ρ, σ)
    abs(1 - real(tr(ρ * σ)) - 2*sqrt(abs(det(ρ)*det(σ))))#^2
end

function fidelity(ρ, σ)
    real(tr(ρ * σ)) + 2*sqrt(abs(det(ρ)*det(σ)))#^2
end

fidelity (generic function with 1 method)

In [5]:
# Define the Pauli matrices
σˣ = [0 1; 1 0]
σʸ = [0 im; -im 0]
σᶻ = [1 0; 0 -1]

# Define the basis elements
fᴷ₁ = σˣ / 2
fᴷ₂ = σʸ / 2
fᴷ₃ = σᶻ / 2

# Check orthogonality and normalization
@assert tr(fᴷ₁ * fᴷ₂) ≈ 0
@assert tr(fᴷ₁ * fᴷ₃) ≈ 0
@assert tr(fᴷ₂ * fᴷ₃) ≈ 0
@assert tr(fᴷ₁ * fᴷ₁) ≈ 1 / 2
@assert tr(fᴷ₂ * fᴷ₂) ≈ 1 / 2
@assert tr(fᴷ₃ * fᴷ₃) ≈ 1 / 2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

# Function to calculate Dc
function Dc(ρ, t, H, C)
    U = (H * ρ - ρ * H) / im
    D = sum(C .* [2 * fᵢ * ρ * fⱼ' - ρ * fⱼ' * fᵢ - fⱼ' * fᵢ * ρ for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ]) / 2
    return U + D
end

Dc (generic function with 1 method)

### Kossakowski objective for two-level system

A linear operator $L: M(2) \rightarrow M(2)$ is the generator of a dynamical semigroup $t \rightarrow \Lambda_t$ of $M(2)$ iff it can be written in the form:

$L = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where

(i) $H = \sum_1^3 h_i f_i, \quad h_i \in \R$

(ii) $f_i f_j = \frac{1}{4}\delta_{ij}I  +\frac{i}{2} \sum_{k=1}^3 \epsilon_{ijk} f_k$ which means:

$ \operatorname{tr}(f_i f_j)=\frac{1}{2}\delta_{ij}, \quad \operatorname{tr}(f_i) = 0$


(iii) $\{C_{ij}\}= \begin{pmatrix} 
\gamma - 2\gamma_1    & - i a_3            &  i a_2 \\
i a_3                 & \gamma - 2\gamma_2 & -i a_1 \\
-i a_2                & i a_1              & \gamma - 2\gamma_3 
\end{pmatrix} =$

$=\begin{pmatrix} 
-\gamma_1 + \gamma_2 + \gamma_3   & - i a_3   &  i a_2 \\
i a_3                 & \gamma_1 - \gamma_2 + \gamma_3 & -i a_1 \\
-i a_2                & i a_1      & \gamma_1 + \gamma_2 - \gamma_3 
\end{pmatrix}$

In [6]:
# Define functions to construct H and C from parameters
function construct_H(ϵ, h_Re, h_Im)
    return [
             ϵ                     h_Re + im * h_Im
             h_Re - im * h_Im     -ϵ
           ] / 2
end

function construct_C(γ, a)
    return [
        -γ[1] + γ[2] + γ[3]  -im * a[3]            im * a[2]
        im * a[3]            γ[1] - γ[2] + γ[3]   -im * a[1]
        -im * a[2]           im * a[1]             γ[1] + γ[2] - γ[3]
    ]
end

construct_C (generic function with 1 method)

In [7]:
# Objective function

function kossak_obj(ϵ, h_Re, h_Im, γ, a, ρ, t)
    H = construct_H(ϵ, h_Re, h_Im)
    C = construct_C(γ, a)
    obj = 0.0
    for i in 3:length(ρ)
        ρ1 = ρ[i]
        ρ2 = ρ[i - 2] + (t[i] - t[i - 1]) * (Dc(ρ[i], t[i], H, C) + 4 * Dc(ρ[i - 1], t[i - 1], H, C) + Dc(ρ[i - 2], t[i - 2], H, C)) / 3
        obj += infidelity_norm(ρ1,ρ2)
        #obj += fidelity(ρ1,ρ2)
    end
    return obj
end

# Define the objective function wrapper
function objectiveGEXY(ϵ, h_Re, h_Im, γ1, γ2, γ3, a1, a2, a3)
    γ = [γ1, γ2, γ3]
    a = [a1, a2, a3]
    objGEXY = kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵍ, tᵍ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρᵉ, tᵉ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρˣ, tˣ) + 
              kossak_obj(ϵ, h_Re, h_Im, γ, a, ρʸ, tʸ)
    return objGEXY
end

objectiveGEXY (generic function with 1 method)

In [8]:
# Define a function to create and solve the model with given initial guesses and a general upper bound
function solve_model(initial_guess, upper_bound)
    model = Model(optimizer_with_attributes(NLopt.Optimizer, "algorithm" => :LD_SLSQP))

    # Register the element-wise power operator
    register(model, :.^, 2, (x, y) -> x.^y, autodiff=true)

    # Register the custom objective function
    register(model, :objectiveGEXY, 9, objectiveGEXY, autodiff = true)

    # Define lower and upper bounds
    lower_bound = -upper_bound

    # Define variables with general upper and lower bounds and initial guesses
    @variable(model, 0 <= ϵ <= upper_bound, start = initial_guess[1])
    @variable(model, lower_bound <= h_Re <= upper_bound, start = initial_guess[2])
    @variable(model, lower_bound <= h_Im <= upper_bound, start = initial_guess[3])
    @variable(model, lower_bound <= γ[i=1:3] <= upper_bound)
    set_start_value.(γ, initial_guess[4:6])
    @variable(model, lower_bound <= a[i=1:3] <= upper_bound)
    set_start_value.(a, initial_guess[7:9])

    # Define κ terms using anonymous construction with bounds
    @variable(model, lower_bound <= κ[1:3] <= upper_bound)

    @constraint(model, κ[1] == -γ[1] + γ[2] + γ[3])
    @constraint(model, κ[2] == γ[1] - γ[2] + γ[3])
    @constraint(model, κ[3] == γ[1] + γ[2] - γ[3])

    # Define the real part of Cˢʸᵐᵇ matrix using fewer constraints
    @variable(model, C_Re[1:3, 1:3], lower_bound = lower_bound, upper_bound = upper_bound)
    @constraint(model, [i=1:3], C_Re[i, i] == κ[i])
    @constraint(model, [i=1:3, j=1:3; i != j], C_Re[i, j] == 0)

    # Define the imaginary part of Cˢʸᵐᵇ matrix using fewer constraints
    @variable(model, C_Im[1:3, 1:3], lower_bound = lower_bound, upper_bound = upper_bound)
    @constraint(model, C_Im[1, 1] == 0)
    @constraint(model, C_Im[2, 2] == 0)
    @constraint(model, C_Im[3, 3] == 0)
    @constraint(model, C_Im[1, 2] == -a[3])
    @constraint(model, C_Im[1, 3] == a[2])
    @constraint(model, C_Im[2, 1] == a[3])
    @constraint(model, C_Im[2, 3] == -a[1])
    @constraint(model, C_Im[3, 1] == -a[2])
    @constraint(model, C_Im[3, 2] == a[1])

    # Define the κ-related constraints
    @constraint(model, sum(κ) >= 0)
    @constraint(model, [i=1:3], κ[i] >= 0)

    # Nonlinear constraints
    @NLconstraint(model, κ[1] * κ[2] + κ[3] * κ[1] + κ[2] * κ[3] - (a[1]^2 + a[2]^2 + a[3]^2) >= 0)
    @NLconstraint(model, κ[1] * κ[2] * κ[3] - (κ[1] * a[1]^2 + κ[2] * a[2]^2 + κ[3] * a[3]^2) >= 0)

    # Objective function
    @NLobjective(model, Min, objectiveGEXY(ϵ, h_Re, h_Im, γ[1], γ[2], γ[3], a[1], a[2], a[3]))

    # Solve the model
    JuMP.optimize!(model)

    # Retrieve and print results
    objective_value = JuMP.objective_value(model)
    ϵ_value = value(ϵ)
    h_Re_value = value.(h_Re)
    h_Im_value = value.(h_Im)
    γ_value = value.(γ)
    a_value = value.(a)
    C_Re_value = value.(C_Re)
    C_Im_value = value.(C_Im)

    #println("Initial Guess: ", initial_guess)
    println("Objective Value: ", objective_value)
    println("ϵ: ", ϵ_value)
    #println("γ: ", γ_value)
    #println("a: ", a_value)
    #println("C_Re: ", C_Re_value)
    #println("C_Im: ", C_Im_value)

    return objective_value, ϵ_value, h_Re_value, h_Im_value, γ_value, a_value
end

solve_model (generic function with 1 method)

In [9]:
# Define the density matrix evolution and time points

file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"
γᶠ = parse(ComplexF64, γᵢ)

tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γᵢ)
tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γᵢ)
tˣ, ρˣ = read_timeevolution(file_name, "B3", γᵢ)
tʸ, ρʸ = read_timeevolution(file_name, "B4", γᵢ);

In [10]:
# Define a general upper bound for all variables
upper_bound = 260

#Smart guesses for pure relaxation
smart_guess = zeros(9)
#smart_guess[1] = 25.133 # ϵ 
#smart_guess[2] = 0.0
#smart_guess[3] = 0.0
smart_guess[4] = γᶠ/2  # γ1
smart_guess[5] = γᶠ/2  # γ2
smart_guess[6] = γᶠ    # γ3

object_val, ϵ_value, h_Re_value, h_Im_value, γ_value, a_value = solve_model(smart_guess, upper_bound)

Objective Value: 0.0021865472838501856
ϵ: 25.142887457244328


(0.0021865472838501856, 25.142887457244328, 0.0002887366572396451, -0.00014725022059899017, [0.11180110426950224, 0.14791208654099877, 0.2596558711767307], [-2.8987961872230898e-6, 2.4452709895436044e-6, -0.2571905999243906])

In [11]:
# Substitute the optimized parameters back into H and C
optimized_H = construct_H(ϵ_value, h_Re_value, h_Im_value)
optimized_C = construct_C(γ_value, a_value)

3×3 Matrix{ComplexF64}:
 0.295767+0.0im             -0.0+0.257191im          0.0+2.44527e-6im
     -0.0-0.257191im    0.223545+0.0im              -0.0+2.8988e-6im
      0.0-2.44527e-6im      -0.0-2.8988e-6im  5.73196e-5+0.0im

In [12]:
optimized_H

2×2 Matrix{ComplexF64}:
     12.5714+0.0im         0.000144368-7.36251e-5im
 0.000144368+7.36251e-5im     -12.5714+0.0im

In [13]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

get_lindblad_operators (generic function with 1 method)

In [14]:
Hˢⁱᵈ = convert.(ComplexF64,optimized_H)
Cˢⁱᵈ = convert.(ComplexF64,optimized_C)

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

2-element Vector{Any}:
 ComplexF64[0.003786545310616228 + 0.0im -7.0900199186497525e-6 - 5.233027469420939e-6im; 0.00010177018500384813 + 7.463414704546579e-5im -0.003786545310616228 + 0.0im]
 ComplexF64[1.8387876212633794e-6 + 0.0im -0.3648445919085248 + 0.3539940660642903im; -0.02548809579250963 + 0.024730076511351257im -1.8387876212633794e-6 + 0.0im]

In [15]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [16]:
using QuantumOptics
basis = NLevelBasis(2)

include("../LiPoSID.jl")

using Statistics

┌ Warning: Package LinearSolve does not have KrylovKit in its dependencies:
│ - If you have LinearSolve checked out for development and have
│   added KrylovKit as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with LinearSolve
└ Loading KrylovKit into LinearSolve from project dependency, future warnings for LinearSolve are suppressed.


In [16]:
function test_D10(Hˢⁱᵈ, effective_Lindblad)
    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert.(Float64, tₛ)

        #Simulated LME 
        #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
        #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

        ρₒ = DenseOperator(basis,ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

        #print("Simulating Kossakowski")

        tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

        #print("Calculating Fidelity")

        #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
        
        FminState = minimum(F)
        FmedianState = median(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)

    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    println()
    println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
    println("Median fidelity for "*γᵢ*": ", F_median_value)

    return(F_min_value, F_median_value)
    
end

test_D10 (generic function with 1 method)

In [17]:
test_D10(Hˢⁱᵈ, effective_Lindblad)

D1 

LoadError: UndefVarError: DenseOperator not defined

In [35]:
γᶠ = real(γᶠ)

0.25133

In [38]:
H_anz = [ 25.133   0+0im
           0.      25.133 ]/2

C_anz = [    γᶠ       im*γᶠ    0.0
            -im*γᶠ      γᶠ      0.0
            0.0        0.0      0.0  ]

ansatz_Lindblad = get_lindblad_operators(C_anz, fᴼᴺᴮ)

test_D10(Hˢⁱᵈ, ansatz_Lindblad)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9993939126448471
Median fidelity for 0.25133: 0.9999685606614921


(0.9993939126448471, 0.9999685606614921)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 

Mimimal fidelity for 0.25133: 0.9995188642141652

Median fidelity for 0.25133: 0.9999049135214064